In [ ]:
import os 
from glob import glob

import random
import pandas as pd
from tqdm import tqdm

In [ ]:
import whisper

model_tiny = whisper.load_model("tiny")
model_base = whisper.load_model("base")
model_small = whisper.load_model('small')
model_medium =whisper.load_model('medium')
model_large = whisper.load_model('large')

In [ ]:
model_medium =whisper.load_model('large')


In [ ]:
def test_audio(audio_file):
    model_types = ['tiny', 'base', 'small', 'medium', 'large']

    for model_type in model_types:
        print("Currently Running " + model_type + " model")
        model = whisper.load_model(model_type)
        # load audio and pad/trim it to fit 30 seconds
        audio = whisper.load_audio(audio_file)
        audio = whisper.pad_or_trim(audio)

        # make log-Mel spectrogram and move to the same device as the model
        mel = whisper.log_mel_spectrogram(audio).to(model.device)

        # detect the spoken language
        _, probs = model.detect_language(mel)
        print(f"Detected language: {max(probs, key=probs.get)}")

        # decode the audio
        options = whisper.DecodingOptions()
        result = whisper.decode(model, mel, options)

        # print the recognized text
        print(result.text)
        print()

# HINDI AUDIO

In [ ]:

df = pd.read_csv('test_audio/hi/train.tsv', sep='\t')

hindi_audio_files = list(df['path'])
hindi_audio = os.path.join('test_audio', 'hi', 'clips',random.choice(hindi_audio_files))

text = df[df['path'] == hindi_audio.split('/')[-1]]['sentence'].iloc[0]

In [ ]:
print(text)

from IPython.display import Audio
Audio(hindi_audio)



In [ ]:
test_audio(hindi_audio)

## PUNJABI AUDIO

In [ ]:
df = pd.read_csv('test_audio/pa-IN/train.tsv', sep='\t')

pb_audio_files = list(df['path'])
pb_audio = os.path.join('test_audio', 'pa-IN', 'clips',random.choice(pb_audio_files))

text = df[df['path'] == pb_audio.split('/')[-1]]['sentence'].iloc[0]

In [ ]:
print(text)

from IPython.display import Audio
Audio(pb_audio)

In [ ]:
test_audio(pb_audio)

# RUN INFERENCE FOR ALL HINDI AUDIOS

In [ ]:
from whisper.tokenizer import LANGUAGES


def get_result(audio_file):
    model_types = ['tiny', 'base', 'small', 'medium', 'large']
    resultDict = {}
    resultDict['file'] = audio_file
    for model_type in model_types:
        # print("Currently Running " + model_type + " model")
        model = whisper.load_model(model_type)
        # load audio and pad/trim it to fit 30 seconds
        audio = whisper.load_audio(audio_file)
        audio = whisper.pad_or_trim(audio)

        # make log-Mel spectrogram and move to the same device as the model
        mel = whisper.log_mel_spectrogram(audio).to(model.device)

        # decode the audio
        # parameters language='hi' tells the decoder that we are currently working for hindi
        options = whisper.DecodingOptions(language='hi') 
        result = whisper.decode(model, mel, options)

        resultDict[model_type] = result.text
    return resultDict

In [ ]:
df = pd.read_csv('test_audio/hi/train.tsv', sep='\t')
df = df.reset_index()  # make sure indexes pair with number of rows

results_df = pd.DataFrame()
for index, row in tqdm(df.iterrows()):
    audio_file_path = row['path']
    audio_file = os.path.join('test_audio','hi', 'clips', audio_file_path)

    text = df[df['path'] == audio_file_path.split('/')[-1]]['sentence'].iloc[0]
    result = get_result(audio_file)
    result['original_text'] = text
    print(result)

    results_df = pd.concat([results_df, pd.DataFrame([result])])
    results_df.to_csv('results.csv')


In [ ]:
df = pd.read_csv('test_audio/hi/train.tsv', sep='\t')
df = df.reset_index()  # make sure indexes pair with number of rows

results_df = pd.DataFrame()
for index, row in tqdm(df.iterrows()):
    audio_file_path = row['path']
    audio_file = os.path.join('test_audio','hi', 'clips', audio_file_path)

    text = df[df['path'] == audio_file_path.split('/')[-1]]['sentence'].iloc[0]
    result = get_result(audio_file)
    result['original_text'] = text
    print(result)

    results_df = pd.concat([results_df, pd.DataFrame([result])])
    results_df.to_csv('results_test.csv')
